In [0]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
import re, string, unicodedata
import nltk
#import contractions
#import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.metrics import f1_score
import math
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

In [0]:
# read in the dataset
df1 = pd.read_csv('train.dat',header=None,sep='\t')
df2 = pd.read_csv('test.dat',header=None,sep='\t')
df2.insert(0,'cls',0)
df2.columns = [0, 1]
df=df1.append(df2,ignore_index=True)


In [0]:
# separate names from classes
vals = df.iloc[:,:].values

In [0]:
#dividing into classes and abstract called as 'names'
cls=[]
names=[]
for i in range(len(vals)):
    names.append(vals[i][1])
    if (vals[i][0])==0:
        continue
    cls.append(vals[i][0])


In [0]:
border=len(cls)

In [0]:
#defining normalising functions

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [0]:
#breaking sentences into words 'tokenizing'
for i in range(len(vals)):
    names[i]=nltk.word_tokenize(names[i])
#print(names[774])

In [0]:
#removing punctuation
for i in range(len(vals)):
    names[i]=remove_punctuation(names[i])
#print(names[963])

In [0]:
#removing stop words like 'the','a' etc
for i in range(len(vals)):
    names[i]=remove_stopwords(names[i])
#print(names[963])

In [0]:
#lemmatizing words
for i in range(len(vals)):
    names[i]=lemmatize_verbs(names[i])
#print(names[963])


In [0]:
#stemming 
for i in range(len(vals)):
    names[i]=stem_words(names[i])
#print(names[963])

In [0]:
#converting into lower case
for i in range(len(vals)):
    names[i]=to_lowercase(names[i])
#print(names[774])

In [0]:
#csr matrix
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
   
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d )
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

In [0]:
#csr information
def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

In [0]:
#normalise csr
def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

In [0]:
def classifyNames(names, cls,k=10):
   
    docs = names
    mat = build_matrix(docs)
    csr_l2normalize(mat)
    train=mat[:border-1,:]
    test=mat[border:,:]
    
    
    def classify(x, train, clstr):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x
        dots = x.dot(train.T)
        #dots[0] = -1 # invalidate self-similarity
        #denom1=denom(x)
        #for i in range(train.shape[0]):
         #   dots[0,i]=dots[0,i]/denom1
          #  y=train[i,:].toarray()[0]
           # denom2=denom(y)
            #dots[0,i]=dots[0,i]/denom2
        
        
        sims = list(zip(dots.indices, dots.data))
        #n = []
        #x1=x.toarray()[0]
        #for i in range(train.shape[0]):
         #   y=train[i,:].toarray()[0]
          #  n.append(cosine_similarity(x1,y))
        if len(sims) == 0:
            # could not find any neighbors
            return '1' if np.random.rand() > 0.5 else '5'
        sims.sort(key=lambda x: x[1], reverse=True)
        #a=sorted(range(len(n)), key=lambda i: n[i], reverse=True)[:k]
       # wsum=0.0
       # wdenom=0.0
       # for i in sims[:k]:
        #    wsum+=clstr[s[0]]*dots[s[0]]
         #   wdenom+=dots[s[0]]
        #clpred=int(wsum/wdenom)
        #-----------------------------------------------------
        #wsum=0.0
        #wdenom=0.0
        #clpred=0.0
        #p=[clstr[s[0]] for s in sims[:k] if s[1] > 0 ]
        #q=[dots[0,s[0]] for s in sims[:k] if s[1] > 0 ]
        #for i in range(k):
          #  wsum+=p[i]*q[i]
         #   wdenom+=q[i]
        #clpred=wsum/wdenom
        tc = Counter(clstr[s[0]] for s in sims[:k]).most_common(5)
        if len(tc) < 6 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            tc[clstr[s[0]]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]
        #if len(tc) < 5 or tc[0][1] > tc[1][1]:
            # majority vote
         #   return tc[0][0]
        # tie break
        #tc = defaultdict(float)
        #for s in a[:k]:
         #   tc[clstr[s[0]]] += s[1]
        #return clpred
        
    macc = 0.0
    
    #for f in range(d):
        # split data into training and testing
     #   train, clstr, test, clste = splitData(mat, cls, f+1, d)
        # predict the class of each test sample
    clspr = [ classify(test[i,:], train, cls) for i in range(test.shape[0]) ]
        # compute the accuracy of the prediction
       # acc = 0.0
        #for i in range(len(clste)):
         #   if clste[i] == clspr[i]:
          #      acc += 1
        #acc /= len(clste)
        #macc += acc
        
    
    #score=f1_score(clste, clspr, average='micro')
        
    return clspr

In [0]:
#classification file
x=classifyNames(names,cls,30)
with open('format1.dat','w') as filehandle:
    for listitem in x:
        filehandle.write('%d\n' % listitem)